In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1


In [45]:
%%writefile app.py

import streamlit as st
import numpy as np
import pandas as pd
from sklearn import preprocessing
import pickle

# load the model
model = pickle.load(open('lstm_model.pkl','rb'))


st.title("Predictive Maintenance of Aircraft Engine")


df = pd.read_csv('FD001.csv')


df.sort_values(['unit number','time in cycles'], inplace=True)

# Function to categorize failure_within_w1 as 1 or 0
rul = pd.DataFrame(df.groupby('unit number')['time in cycles'].max()).reset_index()

rul.columns = ['unit number', 'max']
df = df.merge(rul, on=['unit number'], how='left')

df['RUL'] = df['max'] - df['time in cycles']

df[['unit number','time in cycles','max','RUL']].head()

df.drop('max', axis=1, inplace=True)

w1 = 30
df['failure_within_w1'] = np.where(df['RUL'] <= w1, 1, 0 )


# Using the difference function, we exclude these columns from the normalization process.
df['cycle_norm'] = df['time in cycles']
cols_normalize = df.columns.difference(['unit number','time in cycles','RUL','failure_within_w1'])

# MinMax normalization (from 0 to 1) of sensor data
min_max_scaler = preprocessing.MinMaxScaler()
norm_df = pd.DataFrame(min_max_scaler.fit_transform(df[cols_normalize]),
                             columns=cols_normalize,
                             index=df.index)

#Join the normalized and non-normalized data.
join_df = df[['unit number','time in cycles','RUL','failure_within_w1']].join(norm_df)
df = join_df.reindex(columns = df.columns)

df.head()

target_variable = 'failure_within_w1'
num_cycles = 30

# Function to categorize failure_within_w1 as 1 or 0
def categorize_failure(row):
    if row['failure_within_w1'] == num_cycles:
        return 1
    else:
        return 0

# Apply categorization function
df['failure_category'] = df.apply(categorize_failure, axis=1)

# Group by engine and find the maximum cycle for each engine
max_cycles = df.groupby('unit number')['failure_within_w1'].max().reset_index()

# Merge the maximum cycles back to the original dataframe
df = pd.merge(df, max_cycles, on='unit number', suffixes=('', '_max'))

# Calculate Remaining Useful Life (RUL)
df['RUL'] = df['failure_within_w1_max'] - df['failure_within_w1']

# Sidebar for user inputs
st.sidebar.header('Settings')
selected_unit = st.sidebar.selectbox('Select Engine Unit Number', df['unit number'].unique())
options = df['time in cycles'].unique()
default_start, default_end = options[0], options[-1]
start_time, end_time = st.sidebar.select_slider("Select the time series length:", options=options, value=(default_start, default_end))

st.write("start time:", start_time)
st.write("end time:", end_time)

# Filter data based on user inputs
filtered_df = df[(df['unit number'] == selected_unit) & (df['time in cycles'] >= start_time) & (df['time in cycles'] <= end_time)]

# Display filtered data
st.subheader('Filtered Data')
st.dataframe(filtered_df)

st.subheader('Time Series Plot')
st.line_chart(filtered_df.set_index('time in cycles')['failure_category'])


# Show a summary or insights based on the analysis
st.subheader('Analysis Summary')
st.write("Average RUL for Engine", selected_unit, ":", filtered_df['RUL'].mean())

if not filtered_df['failure_within_w1'].any():
    # Find the first failure cycle for the selected engine ID
    first_failure_cycle = df[(df['unit number'] == selected_unit) & (df['failure_within_w1'] == 1)]['time in cycles'].min()

    if not pd.isnull(first_failure_cycle):
        # Calculate remaining useful life based on the first failure cycle found and the last cycle in the selected time series
        remaining_useful_life_no_failure = first_failure_cycle - filtered_df['time in cycles'].max()
        st.write("Remaining Useful Life:", remaining_useful_life_no_failure)
    else:
        st.write("No failure cycles found in the selected engine ID.")

# Show time in cycles when failure_within_w1 is 1
st.subheader('Displays predicted failure cycles')
cycles_when_failure_1 = filtered_df[filtered_df['failure_within_w1'] == 1]['time in cycles'].tolist()
st.write("predicted failure cycles:", cycles_when_failure_1)


Overwriting app.py


In [46]:
! pip install pyngrok

In [47]:
from pyngrok import ngrok

In [48]:
!wget -q -O - ipv4.icanhazip.com

34.106.64.173


In [49]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.64.173:8501

npx: installed 22 in 2.897s
your url is: https://shiny-coins-scream.loca.lt
2024-01-11 17:50:42.039996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 17:50:42.040069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 17:50:42.041742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 17:50:43.522597: W tensorflow/compiler/tf2tensorrt/utils/p